In [1]:
import math
import cv2 as cv
import numpy as np
import pandas as pd
from glob import glob
from matplotlib import pyplot as plt
from matplotlib.patches import Ellipse
from config import test_dataset_imagename_front,val_dataset_imagename_front


# ------------------ #
from tensorflow.keras import models, Sequential
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Balanceacc

from models import efficientNetV2B0_model, efficientNetV2B3_model
from config import efficientNet_config
from sklearn.utils import shuffle

from utils import plt_heatmap, plt_roccurve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc


import os
# 使用第一張 GPU 卡
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# 判斷膽管方向是 上面 或 下面 
# 用bbox切割完後的影像 分類模型

In [2]:
# 186資料集
# 要分成train 和 validation

training_img_paths     = sorted(glob('../dataset_20230323/training_dataset/cropped_img/*.png'))  
train_img_paths = []           
val_img_paths = []
for img_path in training_img_paths:
    imgname_front = img_path.split("/")[-1].split("_")[0]
    if imgname_front in val_dataset_imagename_front:
        val_img_paths.append(img_path)
    else:
        train_img_paths.append(img_path)

In [3]:
train_img_arrays = []
train_img_labels = []
validation_img_arrays = []
validation_img_labels = []

for train_img_path in train_img_paths:
    img_array = cv.imread(train_img_path)
    img_array = cv.resize(img_array,(224, 224))     # efficientNetv2B0

    # image array
    train_img_arrays.append(img_array)
    # label
    train_img_labels.append([0])

    smoothed_img2_ROTATE_180 = cv.rotate(img_array, cv.ROTATE_180)   # 轉180度
    # image array(rotate 180) 
    train_img_arrays.append(smoothed_img2_ROTATE_180)
    # label(rotate 180)
    train_img_labels.append([1])

# 驗證集
for val_img_path in val_img_paths:
    img_array = cv.imread(val_img_path)
    img_array = cv.resize(img_array,(224, 224))     # efficientNetv2B0


    # image array
    validation_img_arrays.append(img_array)
    # label
    validation_img_labels.append([0])

    smoothed_img2_ROTATE_180 = cv.rotate(img_array, cv.ROTATE_180)
    # image array(rotate 180) 
    validation_img_arrays.append(smoothed_img2_ROTATE_180)
    # label(rotate 180)
    validation_img_labels.append([1])   
    

    # ## plot img
    # plt.figure()
    # f, axarr = plt.subplots(1, 2,figsize=(6,3)) 
    # axarr[0].imshow(cv.cvtColor(smoothed_img2, cv.COLOR_BGR2RGB))
    # axarr[1].imshow(cv.cvtColor(smoothed_img2_ROTATE_180, cv.COLOR_BGR2RGB))
    # plt.show()

In [ ]:
train_img_arrays = np.array(train_img_arrays)
train_img_labels = np.array(train_img_labels)
validation_img_arrays = np.array(validation_img_arrays)
validation_img_labels = np.array(validation_img_labels)

In [ ]:
train_img_arrays, train_img_labels           = shuffle(train_img_arrays,train_img_labels)
validation_img_arrays, validation_img_labels = shuffle(validation_img_arrays,validation_img_labels)
print('訓練集維度= ',train_img_arrays.shape)
print('測試集維度= ',validation_img_arrays.shape)

In [ ]:
model = efficientNetV2B0_model()

model.compile(loss='binary_crossentropy', 
              optimizer=Adam(learning_rate=1e-4),
              metrics=['accuracy', 
                        Balanceacc()
                      ])

In [ ]:
day = '20230409'
checkpoint_filepath = '../model/202303/{}.weights'.format(day)
model_checkpoint_callback = ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_balanceacc',
        mode='max',
        save_best_only=True)

# learning rate 降低
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
#                               factor=0.1,
#                               patience=5, 
#                               min_lr=1e-6)



callbacks = [model_checkpoint_callback, 
             #reduce_lr
             ]

In [ ]:
# 訓練
epochs = 200
history = model.fit(
      x = train_img_arrays,
      y = train_img_labels,
      validation_data = (validation_img_arrays,validation_img_labels),      
      epochs          = epochs,
      verbose         = 1,
      callbacks       = callbacks
)

# 畫圖

In [ ]:
loss         = history.history['loss']
val_loss     = history.history['val_loss']
accuracy     = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
BlA          = history.history['balanceacc']
val_BlA      = history.history['val_balanceacc']

In [ ]:
x = [i for i in range(1, epochs+1)]

min_valloss_x = val_loss.index(min(val_loss)) + 1
min_valloss_y = min(val_loss)

plt.figure(figsize=(24,4))
plt.plot(x, loss, 'r')     
plt.plot(x, val_loss, 'b')     # red dotted line (no marker)

plt.plot(min_valloss_x, min_valloss_y, 'd', color='g')
plt.text(min_valloss_x, min_valloss_y, "({},{})".format(min_valloss_x,round(min_valloss_y,2)), ha='left',va='top',fontsize=20)

plt.legend(['loss','val_loss'])
plt.ylim((0, 1.0))
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('loss')
plt.savefig('../model/202303/{}_loss.png'.format(day),
            bbox_inches='tight',
            pad_inches=1,
            transparent=False)
plt.show()

In [ ]:
x = [i for i in range(1, epochs+1)]

max_valacc_x = val_accuracy.index(max(val_accuracy)) + 1
max_valacc_y = max(val_accuracy)

plt.figure(figsize=(24,4))
plt.plot(x, accuracy, 'r')     
plt.plot(x, val_accuracy, 'b')     # red dotted line (no marker)

plt.plot(max_valacc_x, max_valacc_y, 'd', color='g')
plt.text(max_valacc_x, max_valacc_y, "({},{})".format(max_valacc_x, round(max_valacc_y,2)), ha='left',va='top',fontsize=20)

plt.legend(['accuracy','val_accuracy'])
plt.ylim((0, 1.1))
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('accuracy')
plt.savefig('../model/202303/{}_acc.png'.format(day),
            bbox_inches='tight',
            pad_inches=1,
            transparent=False)
plt.show()

In [ ]:
x = [i for i in range(1, epochs+1)]

max_valBlA_x = val_BlA.index(max(val_BlA)) + 1
max_valBlA_y = max(val_BlA)

plt.figure(figsize=(24,4))
plt.plot(x, BlA, 'r')     
plt.plot(x, val_BlA, 'b')     # red dotted line (no marker)

plt.plot(max_valBlA_x, max_valBlA_y, 'd', color='g')
plt.text(max_valBlA_x, max_valBlA_y, "({},{})".format(max_valBlA_x, round(max_valBlA_y,2)), ha='left',va='top',fontsize=20)

plt.legend(['balance accuracy','val_balance accuracy'])
plt.ylim((0, 1.1))
plt.xlabel('epoch')
plt.ylabel('balance accuracy')
plt.title('balance accuracy')
plt.savefig('../model/202303/{}_BlA.png'.format(day),
            bbox_inches='tight',
            pad_inches=1,
            transparent=False)
plt.show()

In [ ]:
import json
# Get the dictionary containing each metric and the loss for each epoch
history_dict = history.history
# Save it under the form of a json file
json.dump(history_dict, open('../model/202303/{}.history'.format(day), 'w'))

# 分類模型(最後一個epoch模型) 測試集結果

In [ ]:
test_img_paths     = sorted(glob('../dataset_20230323/test_dataset/cropped_img/*.png'))                  # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 換資料集

test_img_arrays = []
test_img_labels = []

for test_img_path in test_img_paths:
    img_array = cv.imread(test_img_path)
    img_array = cv.resize(img_array,(224, 224))     # efficientNetv2B0

    # 兩次雙邊濾波
    # smoothed_img  = cv.bilateralFilter(img_array, 15, 50,  50)
    # smoothed_img2 = cv.bilateralFilter(smoothed_img, 15, 50, 50)
    # image array
    test_img_arrays.append(img_array)
    # label
    test_img_labels.append([0])

    smoothed_img2_ROTATE_180 = cv.rotate(img_array, cv.ROTATE_180)
    # image array(rotate 180) 
    test_img_arrays.append(smoothed_img2_ROTATE_180)
    # label(rotate 180)
    test_img_labels.append([1])


# to numpy array
test_img_arrays = np.array(test_img_arrays)
test_img_labels = np.array(test_img_labels)

# 預測

In [ ]:

# 也將最後一個epoch訓練的模型存下來
model.save("../model/202303/{}_lastepoch.h5".format(day))

pred_result = model.predict(test_img_arrays)

# 結果

In [ ]:
fpr, tpr, thresholds = roc_curve(test_img_labels, pred_result, pos_label=1) 
roc_auc = auc(fpr, tpr)
plt_roccurve(fpr,tpr,roc_auc)

In [ ]:
pred_result = np.where(pred_result > 0.5, 1,0)
tn, fp, fn, tp = confusion_matrix(test_img_labels, pred_result).ravel()
sensitivity = round(tp / (tp+fn), 4)
specificity = round(tn / (tn+fp), 4)
balanced_acc= round((sensitivity+specificity)/2, 4)
precision   = round(tp / (tp+fp), 4)
f1score     = round(2/((1/precision)+(1/sensitivity)), 4)
accuracy    = round((tp+tn)/(tn+fp+fn+tp), 4)

print('Sensitivity= ',sensitivity)
print('Specificity= ',specificity)
print('Balanced_acc= ',balanced_acc)
print('Precision= ', precision)
print('f1score= ', f1score)
print('Accuracy= ',accuracy)

In [ ]:
plt_heatmap([[tp,fn],[fp,tn]])